An issue in JIRA: https://github.com/pallets/jinja/issues/276

Python used to show the first line number in the traceback, but recent change made it show the last line. However, if you wrap a call in `str()` or `repr()`, then it displays the correct line - i.e. it only happens with string concatenation.

The issue is more complicated in Jinja as we cannot actually invoke any of the class methods, like `__str__()` or `__repr__()`, as that would raise the exception. Jinja compiler has some special cases, where the exception is expected to exist, without being raised (pending investigation).

In [2]:
class Z(object):
    def raise_error(self, *args, **kwargs):
        raise ValueError('Z error')
        
    __add__ = __radd__ = __mul__ = __rmul__ = __div__ = __rdiv__ = \
        __truediv__ = __rtruediv__ = __floordiv__ = __rfloordiv__ = \
        __mod__ = __rmod__ = __pos__ = __neg__ = __call__ = \
        __getitem__ = __lt__ = __le__ = __gt__ = __ge__ = __int__ = \
        __float__ = __complex__ = __pow__ = __rpow__ = __sub__ = \
        __rsub__ = raise_error

    __iter__ = __str__ = __len__ = __nonzero__ = __eq__ = \
        __ne__ = __bool__ = __hash__ = raise_error
    

def raise_generator():
    yield 'three values are: %s %s %s' %(
        'primeiro',
        Z(),  # traceback must point to this lineno
        'terceiro'  # but points to this lineno (__str__ only, __eq__ is OK)
    )

In [3]:
print(list(raise_generator()))

ValueError: Z error